# Model Smith-Waterman 

## Import

In [ ]:
import pandas as pd
import posixpath
from data_mining_project import data, preprocessing, DATA_PATH, OUTPUT_PATH, model_sw, validation
import numpy as np
import matplotlib as plt

%matplotlib inline

## Load Data

In [ ]:
file_name = "preprocessed_data.csv"  
file_path = posixpath.join(OUTPUT_PATH, file_name)
data_df = data.load_data_csv(file_path)
data_df.head(5)

In [ ]:
data_df.drop(columns=["ac_dc_prob_num", "ac_dc_prob", "seconds_to_incident_sequence", "dj_ac_state_sequence", "dj_dc_state_sequence", "train_kph_sequence", "ac_dc_prob_timestamp", "hard_braking", "description"], inplace=True)
data_df = data.reformat_str_to_list(data_df, cols=["events_sequence"], col_type=int)
data_df.head(5)

Code for local alignment

In [ ]:
ts = np.array([i/10 for i in range(13, 20)])
f1_scores = []
confusion_matrices = []
for i, t in enumerate(ts):
    filtered_data_df = preprocessing.filter_irrelevant_events(data_df, t)
    filtered_data_df = filtered_data_df[filtered_data_df["events_sequence"].apply(lambda row: preprocessing.remove_short_rows(row, x=2))].reset_index(drop=True)
    data_str_df = filtered_data_df.copy().apply(lambda x: list(map(str, x)))
    XY = filtered_data_df.to_numpy()
    X = XY[:, :-1]
    Y = XY[:, -1]
    print(X.shape, Y.shape)
    print(f"relevance threshold: {t}")
    confusion_matrices.append(model_sw.loo_sw(X, Y))
    f1_score, f1_score_per_class = validation.compute_f1_score(confusion_matrices[i]) 
    print(f"Average f1_score: {f1_score}")
    f1_scores.append((t, f1_score))
    validation.plot_confusion_matrix(confusion_matrices[i], t)

f1_scores = np.array(f1_scores)

## Plot F1-Scores

In [ ]:
validation.plot_f1_scores(f1_scores)